## Preprocess Augmented Data
GPT2로 질의를 생성한 데이터를 dense encoder 학습에 사용할 수 있도록 preprocess

In [1]:
## Load Libraries
import pandas as pd
import numpy as np
import json
import os
import json
from collections import Counter
from tqdm.auto import tqdm
from datasets import load_from_disk, Dataset

In [2]:
## Load wiki data
wiki_path = "wikipedia_documents.json"
with open(os.path.join("../data/", wiki_path), "r", encoding="utf-8") as f:
    wiki = pd.read_json(f, orient='index')

In [3]:
wiki.columns

Index(['text', 'corpus_source', 'url', 'domain', 'title', 'author', 'html',
       'document_id'],
      dtype='object')

In [4]:
# print(wiki.corpus_source.unique())
wiki.drop(["url", "corpus_source", "domain", "author", "html"], axis=1, inplace=True)

In [5]:
## 필요한 column만 남겨둠
wiki.columns = ['context', 'title', 'document_id']
wiki = wiki[["title", "context", "document_id"]]

In [6]:
wiki

,title,context,document_id
0,나라 목록,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",0
1,나라 목록,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,1
2,백남준,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,2
3,아오조라 문고,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",3
4,아오조라 문고,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,4
...,...,...,...
60608,오키나와 현립 박물관·미술관,"오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...",60608
60609,오키나와 현립 박물관·미술관,"1936년 7월, 오키나와현 교육위원회 부설 향토 박물관(沖縄県教育会附設郷土博物館)...",60609
60610,오키나와 현립 박물관·미술관,박물관은 개관 10주년이되는 2017년에 관의 애칭 및 마스코트를 일반인에게 공모했...,60610
60611,단결권 및 단체교섭권 협약,단결권 및 단체교섭권 협약(Right to Organise and Collectiv...,60611


In [8]:
## train_dataset 형태 확인
train_dataset = load_from_disk("../data/train_dataset")
train_dataset

DatasetDict({
    train: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 3952
    })
    validation: Dataset({
        features: ['__index_level_0__', 'answers', 'context', 'document_id', 'id', 'question', 'title'],
        num_rows: 240
    })
})

In [9]:
# Datasetdict to pandas
train_data = Dataset.to_pandas(train_dataset["train"])
valid_data = Dataset.to_pandas(train_dataset["validation"])

In [10]:
## Load augmented data
gen_data = pd.read_csv("../data/generated_data.csv")

In [11]:
## column 명 통일
gen_data.columns = ['context', 'answer', 'question']

In [12]:
# question 중복 데이터 확인
x = Counter(gen_data.question)
# print(x.most_common())
# question 중복 데이터 제거
gen_data.drop_duplicates(subset="question",ignore_index=True,keep="first", inplace=True)
gen_data.dropna(inplace=True)

In [13]:
gen_data

,context,answer,question
0,체코어에는 많은 방언들이 존재하며 거의 대부분은 서로 이해가능하다. 모든 체코인들이...,체코어,체코인이 서로 의사소통할 수 있는 언어는?
1,"하르키우(Харків, 하리코프, 하리꼬브)는 우크라이나 북동부의 도시이고 하르키우...",하르키우,우크라이나 동부의 도시 이름은?
2,펫 숍 보이스(Pet Shop Boys)는 닐 테넌트(Neil Tennant)와 크...,펫 숍 보이스,데이비드 테넌트와 크리스 로가 1983년에 결성한 영국 2인조 힙합 그룹의 이름은?
3,"정상가족 이데올로기에 대해 설명한다.\n\n사회에서 말하는 ‘정상 가족’이란, 아빠...",정상가족 이데올로기,가족형태에 대해 설명하는 이데올로기는 무엇인가?
4,Stronghold: Crusaders Extreme2008년 6월 2일|EU|20...,스트롱홀드: 크루세이더,2002년 8월 30일 출시된 게임의 이름은 무엇인가?
...,...,...,...
26263,해양 착생식물의 생태는 지상 착생식물의 생태와는 다르다. 해양 생태계의 착생식물은 ...,착생식물,해양 생태계에서 가장 흔한 생물은?
26264,"암흑 에너지(暗黑energy, dark energy)는 물리 우주론과 천문학에서, ...",암흑 에너지,우주에 널리 퍼지고 있는 에너지는 무엇인가?
26265,밋 차이반차 피쳇 품헴 (พิเชษฐ์ พุ่มเหม) 으로 빈곤에서 태어났다 부모...,밋 차이반차,밋 차반차피 품헴으로 빈곤에서 태어난 사람은?
26266,"곽동수는 1989년 월간 마이크로소프트웨어 기고를 시작으로 디지털, 경제 분야 평론...",곽동수,1989년 월간 MSW 기고를 시작한 사람은 누구인가?


In [20]:
# gen_data 확인
gen_data.question[1]
for i in range(100, 200):
    # print(gen_data.context[i])
    print(gen_data.question[i], gen_data.answer[i])

기원전 27년 아우구스투스 시대에 알렉산드리아에 도달한 제국의 이름은? 히스파니아 타라코넨시스
갑이 개를 기르며 얻는 가치는 얼마인가? 코스의 정리
아우스터리츠 슈만은 누구의 아들로 태어났는가? 호르스트 슈만
영국의 작가이자 언론인이며 저널리스트인 사람은? 아서 랜섬
공산주의 사상은 공산주의와 무엇이 구분되는가? 장면
이시마키 현의 이름은? 이시노마키시
1750년대 유럽에서 발생한 음악철학에 관한 논쟁을 일컫는 말은? 부퐁 논쟁
테리 로미니스트리의 사장은 누구인가? 돈 모엔
6세기중엽 물길을 지배하던 부족은? 말갈
프롤레타리아 계급이 성립된 해는? 근대
영국의 해외 영토는 어디인가? 영국령 버진아일랜드
ADDIS를 쓴 사람의 직업은? 해리성 장애
아이폰 11 프로의 무광 색상은? LG Q9
태양에서 가장 뜨거운 곳은 어디인가? 광구
조소순이 태어난 곳은? 전호극
밤에 보이는 별을 이르는 말은? 밤하늘
서역 고가 대합실을 설계한 역은? 선양 역
황실 천문대의 요직에 선교사들을 임명한 것은 누구인가? 강희제
중국의 역사 속에서 춘추전국시대에 주요 유파로 분류되었던 사람은? 법가
프랑스 축구의 하위리그는? 리그 2
보통나기나타 갑옷의 무기는? 아시가루
베트남 남부 지역의 세계 야생생물 기금은?  누이쭈아 국립공원
베트남전쟁이 일어난 나라는 어디인가? 라오스
노르웨이 실험음악 록밴드는 누구인가? 울베르
수학에서 가장 역사가 깊은 분야는? 산술
학교교육의 자율화를 위해 정책을 펼친 대통령은 누구인가? 이명박 정부
일본의 철도 애호가 단체의 이름은? 데쓰도토모노카이
프란체스코가 교황으로 선출된 사람은 누구인가? 이폴리토 데 메디치
SDS가 병원 패키지 수출을 위해 다각도로 영업활동을 한 회사는? 삼성SDS
1920∼1960년 대통령 관저는 어디에 사용되었는가? 쿠바
가은읍에서 남서쪽으로 위치한 서원 이름은? 소양서원
오식등을 볼 수 있는 텍스투스의 첫번째 판은? 텍스투스 레셉투스
사랑에 관한 많은 연구를 진행한 인류학자는 누구인가? 사랑
세르비아가 승리한 전투의 이름은?

In [15]:
# wiki 데이터 내 context 중복 데이터 확인
x = Counter(wiki.context)
# print(x.most_common())
# question 중복 데이터 제거
wiki.drop_duplicates(subset="context",ignore_index=True,keep="first", inplace=True)  # 60613 -> 56737
wiki.dropna()

,title,context,document_id
0,나라 목록,"이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개...",0
1,나라 목록,이 목록에 실린 국가 기준은 1933년 몬테비데오 협약 1장을 참고로 하였다. 협정...,1
2,백남준,현 서울특별시 종로구 서린동 (구 일제 강점기 경기도 경성부 서린정) 출신이다. 친...,2
3,아오조라 문고,"아오조라 문고(靑空文庫, あおぞらぶんこ|아오조라 분고)는 ‘일본어판 구텐베르크 프로...",3
4,아오조라 문고,저자 사망 이후 50년이 지나 저작권이 소멸한 메이지 시대부터 쇼와 시대 초기까지의...,4
...,...,...,...
56732,오키나와 현립 박물관·미술관,"오키나와 현립 박물관·미술관( , Okinawa Prefectural Museum ...",60608
56733,오키나와 현립 박물관·미술관,"1936년 7월, 오키나와현 교육위원회 부설 향토 박물관(沖縄県教育会附設郷土博物館)...",60609
56734,오키나와 현립 박물관·미술관,박물관은 개관 10주년이되는 2017년에 관의 애칭 및 마스코트를 일반인에게 공모했...,60610
56735,단결권 및 단체교섭권 협약,단결권 및 단체교섭권 협약(Right to Organise and Collectiv...,60611


In [17]:
# gen_data와 wiki data merge
gen_wiki = pd.merge(gen_data, wiki)  # 26267
sum(gen_wiki.isnull().values)  # null 값 확인

array([0, 0, 0, 0, 0])

### gen_wiki에는 (question에 매칭되는)ID가 없다!

In [35]:
train_data

,title,context,question,id,answers,document_id,__index_level_0__
0,미국 상원,미국 상의원 또는 미국 상원(United States Senate)은 양원제인 미국...,대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?,mrc-1-000067,"{'answer_start': [235], 'text': ['하원']}",18293,42
1,인사조직관리,'근대적 경영학' 또는 '고전적 경영학'에서 현대적 경영학으로 전환되는 시기는 19...,현대적 인사조직관리의 시발점이 된 책은?,mrc-0-004397,"{'answer_start': [212], 'text': ['《경영의 실제》']}",51638,2873
2,강희제,강희제는 강화된 황권으로 거의 황제 중심의 독단적으로 나라를 이끌어 갔기에 자칫 전...,강희제가 1717년에 쓴 글은 누구를 위해 쓰여졌는가?,mrc-1-000362,"{'answer_start': [510], 'text': ['백성']}",5028,230
3,금동삼존불감,"불상을 모시기 위해 나무나 돌, 쇠 등을 깎아 일반적인 건축물보다 작은 규모로 만든...",11~12세기에 제작된 본존불은 보통 어떤 나라의 특징이 전파되었나요?,mrc-0-001510,"{'answer_start': [625], 'text': ['중국']}",34146,992
4,계사명 사리구,동아대학교박물관에서 소장하고 있는 계사명 사리구는 총 4개의 용기로 구성된 조선후기...,명문이 적힌 유물을 구성하는 그릇의 총 개수는?,mrc-0-000823,"{'answer_start': [30], 'text': ['4개']}",47334,548
...,...,...,...,...,...,...,...
3947,이오 (위성),이오의 산\n이오의 산 목록\n 이오에는 100~150개의 산이 있다. 이들 산의 ...,대부분의 이오의 산이 형성되는데 영향을 끼친 것은?,mrc-0-005285,"{'answer_start': [317], 'text': ['지질 구조']}",8275,3445
3948,내니 다이어리,애니의 고군분투 뉴욕 입성기!!\n\n인류학자가 꿈인 21살 소녀 '애니(스칼렛 요...,애니는 어디서 태어났는가?,mrc-1-000699,"{'answer_start': [616], 'text': ['뉴저지']}",37764,450
3949,빈 필하모니 관현악단,1842년에 작곡가이자 지휘자인 오토 니콜라이가 빈 궁정 오페라극장 소속 관현악단을...,1854년에 니콜라이의 뒤를 이어 상임 지휘자로서 활동한 인물은?,mrc-0-003429,"{'answer_start': [255], 'text': ['칼 에케르트']}",12730,2252
3950,드래곤,원어는 고대 그리스어까지 거슬러 올라간다. 영어 문헌에 이 말이 나타나기 시작한 것...,드래곤의 암수 구분이 있는 신화는?,mrc-0-003956,"{'answer_start': [476], 'text': ['슬라브 신화']}",10924,2595


In [53]:
## train_data의 ID 값 특성 확인
min(train_data["id"].map(lambda x: int(x[7::]))) 
max(train_data["id"].map(lambda x: int(x[7::])))  # id의 마지막 숫자는 0 ~ 5513 의 값

5513

gen_wiki의 document id를 활용해 -> 임의로 ID 값을 추가해주자! 단, 원 training set + gen_wiki 내에서도 겹치지 않아야 함

In [62]:
x = Counter(gen_wiki.document_id)
# x.most_common  # 모두 1 -> 겹치는 거 없음!

In [19]:
# mrc-3-document_id 로 임의로 question id 생성
gen_wiki["id"] = gen_wiki["document_id"].map(lambda x: "mrc-3-"+f"{x:06d}")

In [20]:
## column 명 통일
gen_wiki.columns = ['context', 'answers', 'question', 'title', 'document_id', 'id']

### 학습에 활용할 gen_wiki 데이터 필터링  
원 training set 분포와 비슷하게! 

In [21]:
gen_wiki = gen_wiki[gen_wiki["context"].map(lambda x: len(x)) < 2800]  # context 2800자 이상 제거

In [ ]:
sum(train_data["question"].map(lambda x: len(x))) / len(train_data)  # 평균 29
max(train_data["question"].map(lambda x: len(x)))  # max 78
min(train_data["question"].map(lambda x: len(x)))  # min 8

8

In [22]:
gen_wiki = gen_wiki[gen_wiki["question"].map(lambda x: len(x)) >= 10]  # 질문 10자 미만 제거
gen_wiki = gen_wiki[gen_wiki["question"].map(lambda x: len(x)) <= 45]  # 질문 45자 초과 제거

In [24]:
gen_wiki = gen_wiki[["title", "context", "question", "id", "answers", "document_id"]]
gen_wiki  # train_data와 index_level 제외하고 똑같은 형태로 데이터 생성! -> 총 25079개

,title,context,question,id,answers,document_id
0,체코어,체코어에는 많은 방언들이 존재하며 거의 대부분은 서로 이해가능하다. 모든 체코인들이...,체코인이 서로 의사소통할 수 있는 언어는?,mrc-3-001362,체코어,1362
1,하르키우,"하르키우(Харків, 하리코프, 하리꼬브)는 우크라이나 북동부의 도시이고 하르키우...",우크라이나 동부의 도시 이름은?,mrc-3-003477,하르키우,3477
3,정상가족 이데올로기,"정상가족 이데올로기에 대해 설명한다.\n\n사회에서 말하는 ‘정상 가족’이란, 아빠...",가족형태에 대해 설명하는 이데올로기는 무엇인가?,mrc-3-039302,정상가족 이데올로기,39302
4,스트롱홀드: 크루세이더,Stronghold: Crusaders Extreme2008년 6월 2일|EU|20...,2002년 8월 30일 출시된 게임의 이름은 무엇인가?,mrc-3-046543,스트롱홀드: 크루세이더,46543
5,하인리히 쿤라트,하인리히 쿤라트(Heinrich Khunrath; Henricus Khunrath|...,1560년대 독일의 의사이자 연금술사가 된 사람은?,mrc-3-056299,하인리히 쿤라트,56299
...,...,...,...,...,...,...
26263,착생식물,해양 착생식물의 생태는 지상 착생식물의 생태와는 다르다. 해양 생태계의 착생식물은 ...,해양 생태계에서 가장 흔한 생물은?,mrc-3-051188,착생식물,51188
26264,암흑 에너지,"암흑 에너지(暗黑energy, dark energy)는 물리 우주론과 천문학에서, ...",우주에 널리 퍼지고 있는 에너지는 무엇인가?,mrc-3-019928,암흑 에너지,19928
26265,밋 차이반차,밋 차이반차 피쳇 품헴 (พิเชษฐ์ พุ่มเหม) 으로 빈곤에서 태어났다 부모...,밋 차반차피 품헴으로 빈곤에서 태어난 사람은?,mrc-3-055097,밋 차이반차,55097
26266,곽동수,"곽동수는 1989년 월간 마이크로소프트웨어 기고를 시작으로 디지털, 경제 분야 평론...",1989년 월간 MSW 기고를 시작한 사람은 누구인가?,mrc-3-035013,곽동수,35013


In [97]:
## Convert pandas into Dataset
gen_wiki_dataset = Dataset.from_pandas(gen_wiki)

In [98]:
gen_wiki_dataset  # index_level은 알아서 datasetdict가 정해주는 거 -> 이왕 한 거 validation 도 나누자!

Dataset({
    features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
    num_rows: 25079
})

In [99]:
gen_wiki_split = gen_wiki_dataset.train_test_split()  # 0.75 : 0.25
gen_wiki_split["validation"] = gen_wiki_split["test"]
del gen_wiki_split["test"]  # 이름 바꾸기 위해
gen_wiki_split

DatasetDict({
    train: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 18809
    })
    validation: Dataset({
        features: ['title', 'context', 'question', 'id', 'answers', 'document_id', '__index_level_0__'],
        num_rows: 6270
    })
})

In [100]:
# gen_wiki dataset 파일 저장
gen_wiki_split.save_to_disk("../data/gen_wiki")